In [2]:
# 6.1 - Importação e visualização do banco de dados
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(0)

train = pd.read_csv("../input/titanic/train.csv")
test = pd.read_csv("../input/titanic/test.csv")

train.describe()

In [4]:
test.describe()

In [5]:
train.info()

In [30]:
print('Dados faltantes no treino:')
print(train.isnull().sum())
trainNum = train.select_dtypes(['number']).dropna()

print('\nDados faltantes no teste:')
print(test.isnull().sum())
testNum = test.select_dtypes(['number']).fillna(0)

In [48]:
# 6.2 - Exploração do banco de dados
sns.barplot(data=trainNum, x="SibSp", y="Survived")
plt.ylabel('Sobrevivência')
plt.xlabel('N° de Irmãos e Cônjugues')

# Nota-se que maiores números de cônjujes e irmãos diminui a probabilidade de sobrevivência.

In [49]:
sns.barplot(data=trainNum, x="Parch", y="Survived")
plt.ylabel('Sobrevivência')
plt.xlabel('N° de Pais e Filhos')

# Nota-se maior sobrevivência entre 2 e 4 familiares (1+N°)

In [45]:
bins = list(range(0,81,10))
labels = []
ant = '0'
for i in bins[1:]:
    prox = str(i)
    labels+=['['+ant+','+prox+')']
    ant=prox 
trainAge = trainNum.copy()
trainAge['faixaIdades'] = pd.cut(trainNum['Age'], bins=bins, labels=labels, right=False)
sns.barplot(data=trainAge, x='faixaIdades', y="Survived")

# Os mais idosos têm menos chance de sobrevivência e
# crianças com menos de 10 anos têm a maior.

In [58]:
# 6.3 - Treinamento do modelo e submissão das predições

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X = trainNum.drop(['PassengerId','Survived'],axis=1)
y = trainNum['Survived']

Xtrain,Xval,ytrain,yval = train_test_split(X,y,test_size=0.20,random_state=0)

Xtest = testNum.drop(['PassengerId'],axis=1)

In [75]:
modelo = RandomForestClassifier(n_estimators=150,max_depth=7,random_state=0)
modelo.fit(Xtrain,ytrain)
prev = modelo.predict(Xval)
AC = accuracy_score(prev,yval)
print('A acurácia foi de:',AC)

In [76]:
ID = testNum['PassengerId']
prevt = modelo.predict(Xtest)

submission = pd.DataFrame({'PassengerId' : ID,'Survived' : prevt })
submission.to_csv('submission.csv',index = False)